In [ ]:
import easyidp as idp
from pathlib import Path
from collections import defaultdict
idp.__version__

In [ ]:
## Define input files
# 'roi' contains the geospatial polygons defining the location of your features of interest in 2D space (must be a shapefile)
roi_path = Path("/ofo-share/str-disp_drone-data-partial/taos/emerald-120m_20230401T2215_idptaos.shp")
# the digital surface model (absolute elevations) from the metashape project containing the camera locations
dsm_path = Path("/ofo-share/str-disp_drone-data-partial/str-disp_drone-data_imagery-processed/outputs/flattened-120m/emerald-120m_20230401T2215_dsm_utm.tif")

## Load roi and dsm, use dsm to get elevations of the vertices of the polygons defining features (to be able to reproject in 3D space)
roi = idp.ROI(roi_path, name_field = "treeID")
roi.get_z_from_dsm(dsm_path, mode="point")
roi

In [ ]:
## Load the metashape project containing the camera positions
msproj = Path("/ofo-share/str-disp_drone-data-partial/str-disp_drone-data_imagery-processed/projects/flattened-120m/emerald-120m_20230401T2215.psx")
ms = idp.Metashape(msproj, chunk_id=0)

In [ ]:
## Compute the location of each feature (provided tree polygons) on each photo and save to json
img_dict_ms = roi.back2raw(ms)

# Invert the dict so the top level of the nesting is the photo files, and under them, ROIs (tree polygons)
flipped_dict = defaultdict(dict)
for key, val in img_dict_ms.items():
    for subkey, subval in val.items():
        flipped_dict[subkey][key] = subval

idp.jsonfile.dict2json(flipped_dict, json_path = "/ofo-share/str-disp_drone-data-partial/emerald/tao-img-dict.json", indent = 2)

In [ ]:
## Optional for visualization: draw the location of a specified feature polygon on each photo containing it
ms.show_roi_on_img(img_dict_ms, "644", save_as="/ofo-share/str-disp_drone-data-partial/cropped-photos2/overview_644.png")

# or on a single photo
ms.show_roi_on_img(img_dict_ms, "4", "120m--101MEDIA--DJI_0495.JPG", save_as="/ofo-share/str-disp_drone-data-partial/cropped-photos2/overview_onephoto.png")